In [108]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image as im
import numpy as np
import os
from sklearn.utils import Bunch
scale_percent = 25
s = ['top','middle','bottom']
image_count = []
total_image_count = 0
for folder in s:
    count = 0
    for image_index in range(23):
        path = f'cropped/{folder}/200_100/{image_index:03}/'
        if not os.path.exists(path):
            continue
        
        for file_name in os.listdir(path):
            if file_name.endswith(".jpg"):
                count += 1
    image_count.append(count)
    total_image_count += count

# sliding window parameters set up
sliding_window_size = 200
data = np.zeros([total_image_count, int(sliding_window_size*scale_percent*0.01), int(sliding_window_size*scale_percent*0.01)])
labels = np.zeros(total_image_count)
count = 0
for label, folder in enumerate(s):
    images = []
    for image_index in range(23):
        path = f'cropped/{folder}/200_100/{image_index:03}/'
        if not os.path.exists(path):
            continue

        for file_name in os.listdir(path):
            if file_name.endswith(".jpg"):
#                 image = Image.open(f'{path}{file_name}')
                image = im.imread(f'{path}{file_name}')
                #trans into grayscale
                rgb_weights = [0.2989, 0.5870, 0.1140]
                image = np.dot(image[...,:3], rgb_weights)
#                 images.append(np.array(image))
                images.append(resizeImage(image, scale_percent))
    data[count:count + image_count[label]] = np.array(images)
    labels[count:count + image_count[label]] = np.ones(len(images)) * label
    count += image_count[label]
    
    data = data.astype(np.uint8)
    labels = labels.astype(np.uint8)
# check = image_count[0] + image_count[1]
# print(labels[check])
# print(data[check].shape)
# plt.imshow(data[check], cmap='gray')
    
n = 100; #image numbers
# images=np.zeros(shape=(2*n, sliding_window_size, sliding_window_size))
# labels=np.zeros(shape=(2*n,1))
# for j in range(0,2): 
#     for i in range(1,n+1):
#         filename = 'C:/Users/KLAB/pictures/CNN/'+s[j]+str(i)+'.PNG'
#         image= Image.open(filename)
#         images[i-1+n*j]=np.array(image)
#         labels[i-1+n*j]=j
#         print(filename)
#         plt.imshow(images[i-1],cmap='gray')

np.savez(str(total_image_count)+'_datasetC.npz', data=data, label=labels)

In [101]:
import cv2
def resizeImage(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return resized